# 3d mitochondria visualizer using vtk 
- Includes a Neuroglancer link generator and a vtk viewer centered on a mitoid of interest 
- This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/pni_mito_cellswskel_v185_fullstats.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
0,1783325,527936,75186,43844,1430,74824,43668,1323,75472,44120,1561,648518346349523993,300744,175376,57200,6.187842,20.482515,Basal
1,3272360,354680,101562,47000,241,101260,46750,191,101750,47228,303,648518346349538711,406248,188000,9640,3.780227,10.346855,Apical
2,1776789,523452,74500,43210,1457,73444,42632,1401,75636,43732,1564,648518346349538102,298000,172840,58280,6.992364,30.063988,Apical
3,2158922,4428,81214,54992,1230,81172,54962,1228,81254,55018,1235,648518346349538730,324856,219968,49200,0.129240,2.652803,Apical
4,2158896,11196,81194,53124,2007,81142,53096,2004,81250,53168,2014,648518346349532295,324776,212496,80280,0.224896,2.186498,Axonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210970,1302900,2116324,65836,40492,311,65132,40146,191,66430,40734,368,648518346349516055,263344,161968,12440,20.038195,43.285222,Unknown
210971,3189781,2193980,98800,58132,469,98240,57710,379,99286,58468,536,648518346349539215,395200,232528,18760,21.290737,48.309769,Unknown
210972,2657020,2453896,88816,68882,429,88220,68540,21,89432,69438,877,648518346349539376,355264,275528,17160,25.841965,69.054582,Basal
210973,1307970,2617596,66122,41208,293,65406,40902,173,66600,41576,356,648518346349516055,264488,164832,11720,27.147893,70.360888,Unknown


In [6]:
pyr = pd.read_csv('data/pycids_v185.csv', names=['cellid'], header=None)
pyr.head()

,cellid
0,648518346349539333
1,648518346349539334
2,648518346349523975
3,648518346349539336
4,648518346349538312


In [7]:
pyr_list = pyr.cellid.to_list()
print(pyr_list)

[648518346349539333, 648518346349539334, 648518346349523975, 648518346349539336, 648518346349538312, 648518346349538317, 648518346349523981, 648518346349537297, 648518346349539347, 648518346349537300, 648518346349532180, 648518346349523993, 648518346349538332, 648518346349539359, 648518346349535264, 648518346349538336, 648518346349539366, 648518346349539368, 648518346349533227, 648518346349539376, 648518346349537331, 648518346349535286, 648518346349531192, 648518346349536312, 648518346349538365, 648518346349539390, 648518346349538368, 648518346349535300, 648518346349533252, 648518346349539401, 648518346349538378, 648518346349534289, 648518346349533267, 648518346349538387, 648518346349496405, 648518346349539414, 648518346349523030, 648518346349534292, 648518346349538391, 648518346349538395, 648518346349531230, 648518346349539423, 648518346349524063, 648518346349536354, 648518346349537380, 648518346349503588, 648518346349539430, 648518346349537385, 648518346349539433, 648518346349539435,

In [8]:
# find index number for a pyramidal cellid of interest

pyr.index[pyr.cellid == 648518346349539100]

# 648518346349537741 index 165
# 648518346349538440 index 68
# 648518346349539853 index 195
# 648518346349537297 index 7
# 648518346349539934 index 237
# 648518346349539100 index 301 highest number of unknown designated mitos in a pyr neuron n=150

Int64Index([301], dtype='int64')

# Use index value to view a pyr neuron from list 

In [9]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 301
cellid = pyr_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
29472,1452999,6274288,79456,57454,1580,67776,53814,1513,92092,61680,1636,648518346349539100,317824,229816,63200,72.154369,229.925522,Unknown
17931,2445855,5416184,91810,72054,1231,84884,66526,948,99096,76120,1371,648518346349539100,367240,288216,49240,57.996182,160.228430,Basal
5022,1210754,4281408,73226,50160,322,63978,47728,204,81268,50936,538,648518346349539100,292904,200640,12880,49.218792,156.728437,Unknown
18813,3267298,4046352,101908,47092,633,100708,44386,21,103180,51350,1233,648518346349539100,407632,188368,25320,44.409889,128.896140,Unknown
4616,2161989,4032384,87446,55306,1643,79816,53894,1484,93586,56114,1847,648518346349539100,349784,221224,65720,48.202137,165.959954,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17012,3309060,2336,100480,61950,1503,100466,61916,1499,100502,61968,1508,648518346349539100,401920,247800,60120,0.091322,3.362825,Somatic
16963,2245196,2324,81734,42418,217,81720,42386,212,81754,42446,222,648518346349539100,326936,169672,8680,0.086747,2.912149,Unknown
16385,3417248,2204,103598,62746,1534,103578,62726,1532,103624,62760,1538,648518346349539100,414392,250984,61360,0.078072,2.360419,Somatic
15847,3410881,2096,101930,60590,1468,101900,60554,1465,101952,60624,1471,648518346349539100,407720,242360,58720,0.077584,2.561315,Somatic


In [10]:
cellid

648518346349539100

In [11]:
mito.compartment.unique()

array(['Basal', 'Apical', 'Axonal', 'Unknown', 'Somatic',
       'Unknown dendritic'], dtype=object)

In [12]:
mito_querydf.compartment.unique()

array(['Somatic', 'Unknown', 'Basal', 'Axonal'], dtype=object)

In [13]:
mito_query_basal = mito_querydf[mito_querydf.compartment == 'Basal']
mito_query_basal

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
1988,3859902,259368,111770,61294,1754,111430,61088,1742,111982,61534,1768,648518346349539100,447080,245176,70160,2.894681,8.687554,Basal
3039,3649414,169496,107518,65902,1803,107130,65648,1767,107874,66096,1840,648518346349539100,430072,263608,72120,2.445745,12.269952,Basal
3263,3624223,563944,106622,57314,146,106008,56634,21,107260,58022,251,648518346349539100,426488,229256,5840,7.302811,29.507091,Basal
5343,3088275,39796,96852,61384,2098,96764,61264,2091,96928,61488,2106,648518346349539100,387408,245536,83920,0.609640,3.447231,Basal
7568,3860288,5344,111980,60882,1724,111956,60854,1722,112012,60916,1727,648518346349539100,447920,243528,68960,0.119892,1.454001,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204272,3543379,2579668,105162,73380,670,104568,69534,458,105688,76256,877,648518346349539100,420648,293520,26800,25.945324,63.237962,Basal
207853,4126972,478472,114680,76260,1159,114090,75744,1144,115130,76718,1189,648518346349539100,458720,305040,46360,4.794546,11.599988,Basal
208276,3414671,501892,103334,61476,753,103142,61352,625,103560,61664,849,648518346349539100,413336,245904,30120,6.413595,25.235521,Basal
208658,3782056,1827316,108678,76674,1440,108210,73898,1397,108972,79016,1486,648518346349539100,434712,306696,57600,18.364977,44.696351,Basal


In [14]:
mito_query_apical = mito_querydf[mito_querydf.compartment == 'Apical']
mito_query_apical

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment


In [15]:
mito_query_axonal = mito_querydf[mito_querydf.compartment == 'Axonal']
mito_query_axonal

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
6608,3535190,159204,104000,67610,1860,103802,67350,1843,104138,68098,1879,648518346349539100,416000,270440,74400,2.647779,17.646771,Axonal
45226,3540264,11356,104154,68136,1879,104124,68082,1877,104180,68200,1882,648518346349539100,416616,272544,75160,0.223780,2.093833,Axonal
56873,3672795,15960,106002,76790,2027,105936,76736,2025,106072,76858,2032,648518346349539100,424008,307160,81080,0.298200,2.508330,Axonal
58951,3550588,16804,104940,71778,1919,104894,71680,1915,104988,71856,1923,648518346349539100,419760,287112,76760,0.422323,6.427422,Axonal
60967,3540254,17628,104178,68452,1880,104118,68378,1879,104232,68542,1883,648518346349539100,416712,273808,75200,0.321000,2.564709,Axonal
64418,3672829,19088,106010,76834,2034,105960,76786,2030,106064,76886,2039,648518346349539100,424040,307336,81360,0.294208,1.684115,Axonal
75178,3540241,23832,104164,68392,1883,104108,68280,1879,104208,68534,1888,648518346349539100,416656,273568,75320,0.514770,5.786903,Axonal
77219,3555836,24752,105746,75398,1994,105682,75308,1991,105796,75490,1999,648518346349539100,422984,301592,79760,0.407333,2.658014,Axonal
82324,3421644,27172,103082,64868,1761,103026,64758,1754,103132,64996,1769,648518346349539100,412328,259472,70440,0.500018,4.079828,Axonal
85087,3540210,28580,104216,68730,1890,104078,68588,1880,104352,68862,1894,648518346349539100,416864,274920,75600,0.597699,6.298712,Axonal


In [16]:
mito_query_somatic = mito_querydf[mito_querydf.compartment == 'Somatic']
mito_query_somatic

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
35,3417448,33340,103816,62974,1608,103760,62878,1601,103894,63146,1620,648518346349539100,415264,251896,64320,0.747189,9.042512,Somatic
694,3411689,16104,102770,61410,1719,102696,61276,1715,102848,61552,1722,648518346349539100,411080,245640,68760,0.406888,6.258737,Somatic
1443,3521404,544628,105400,61336,1423,104128,61220,1408,106056,61418,1456,648518346349539100,421600,245344,56920,7.337689,32.092688,Somatic
4615,3304393,177736,100892,61580,1635,100628,61272,1607,101238,61890,1657,648518346349539100,403568,246320,65400,2.483685,11.686035,Somatic
4763,3304284,54516,100670,61060,1573,100558,60844,1563,100762,61284,1584,648518346349539100,402680,244240,62920,0.959285,7.156868,Somatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209726,3416433,624764,103014,62414,1271,102654,62094,1114,103196,62678,1359,648518346349539100,412056,249656,50840,8.984891,44.774903,Somatic
209742,3308919,626656,100712,61978,1605,100334,61456,1578,101382,62546,1625,648518346349539100,402848,247912,64200,7.661802,27.597045,Somatic
210207,3410791,706696,102232,60314,1482,101802,60064,1465,102602,60588,1506,648518346349539100,408928,241256,59280,6.691491,14.455452,Somatic
210521,3416663,807256,103220,62292,1400,102948,61622,1279,103630,62890,1498,648518346349539100,412880,249168,56000,9.823682,35.053269,Somatic


In [17]:
mito_query_unknown = mito_querydf[mito_querydf.compartment == 'Unknown']
mito_query_unknown

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
385,2866447,5392,92254,61260,1570,92226,61242,1568,92292,61280,1574,648518346349539100,369016,245040,62800,0.140131,2.280526,Unknown
2281,2552073,2707920,88992,71546,261,87024,69114,21,91456,74020,521,648518346349539100,355968,286184,10440,27.741793,70.155235,Unknown
4247,3403453,98828,103754,56872,1570,103700,56588,1560,103820,57106,1579,648518346349539100,415016,227488,62800,1.386832,6.580212,Unknown
4616,2161989,4032384,87446,55306,1643,79816,53894,1484,93586,56114,1847,648518346349539100,349784,221224,65720,48.202137,165.959954,Unknown
5022,1210754,4281408,73226,50160,322,63978,47728,204,81268,50936,538,648518346349539100,292904,200640,12880,49.218792,156.728437,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202124,3850960,911312,112422,55706,1369,110948,55442,1297,113862,56230,1456,648518346349539100,449688,222824,54760,9.882040,27.998437,Unknown
203032,3964317,341684,113100,55766,2113,112718,55470,2071,113694,56124,2157,648518346349539100,452400,223064,84520,3.862700,11.894653,Unknown
203500,2739785,351204,90522,53324,2098,90322,53148,2041,90792,53482,2157,648518346349539100,362088,213296,83920,4.199396,14.466705,Unknown
209798,3738582,634508,108970,56552,2016,108832,56190,1895,109148,56876,2157,648518346349539100,435880,226208,80640,7.038631,20.869818,Unknown


In [18]:
mito_query_unkdend = mito_querydf[mito_querydf.compartment == 'Unknown dendritic']
mito_query_unkdend

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment


In [19]:
mito_query_basal_mitolist = mito_query_basal.mito_id.to_list()
print(f"number of basal mitochondria: "+str(len(mito_query_basal_mitolist)))
# print(f"type: "+str(type(mito_query_basal_mitolist)))
# print('')
# print(mito_query_basal_mitolist) # uncomment to print the mito id list

mito_query_apical_mitolist = mito_query_apical.mito_id.to_list()
print(f"number of apical mitochondria: "+str(len(mito_query_apical_mitolist)))
#print(f"type: "+str(type(mito_query_apical_mitolist)))
#print('')
# print(mito_query_apical_mitolist) # uncomment to print the mito id list

mito_query_axonal_mitolist = mito_query_axonal.mito_id.to_list()
print(f"number of axonal mitochondria: "+str(len(mito_query_axonal_mitolist)))
#print(f"type: "+str(type(mito_query_axonal_mitolist)))
#print('')
# print(mito_query_axonal_mitolist) # uncomment to print the mito id list

mito_query_somatic_mitolist = mito_query_somatic.mito_id.to_list()
print(f"number of somatic mitochondria: "+str(len(mito_query_somatic_mitolist)))
#print(f"type: "+str(type(mito_query_somatic_mitolist)))
#print('')
# print(mito_query_somatic_mitolist) # uncomment to print the mito id list

mito_query_unknown_mitolist = mito_query_unknown.mito_id.to_list()
print(f"number of unknown mitochondria: "+str(len(mito_query_unknown_mitolist)))
#print(f"type: "+str(type(mito_query_unknown_mitolist)))
#print('')
# print(mito_query_unknown_mitolist) # uncomment to print the mito id list

mito_query_unkdend_mitolist = mito_query_unkdend.mito_id.to_list()
print(f"number of unknown dendritic mitochondria: "+str(len(mito_query_unkdend_mitolist)))
#print(f"type: "+str(type(mito_query_unkdend_mitolist)))
#print('')
# print(mito_query_unkdend_mitolist) # uncomment to print the mito id list

number of basal mitochondria: 122
number of apical mitochondria: 0
number of axonal mitochondria: 14
number of somatic mitochondria: 245
number of unknown mitochondria: 150
number of unknown dendritic mitochondria: 0


# vtk 3d viewer

In [20]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [21]:
cell_id = cellid

In [22]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [23]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

- Note: The first time you download the meshes will take a long time, which gets longer the more mitos are in the cell of interest
- After downloading the meshes, this cell will run much faster subsequently

In [24]:
# download the mito meshes by compartment for this cell

for i in range(len(mito_query_basal_mitolist)):
    mito_id = mito_query_basal_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_apical_mitolist)):
    mito_id = mito_query_apical_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

for i in range(len(mito_query_axonal_mitolist)):
    mito_id = mito_query_axonal_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_somatic_mitolist)):
    mito_id = mito_query_somatic_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_unknown_mitolist)):
    mito_id = mito_query_unknown_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_unkdend_mitolist)):
    mito_id = mito_query_unkdend_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [25]:
mito_query_basal_mitolist[0]

3859902

In [26]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_basal = {}
for i in range(len(mito_query_basal_mitolist)):
    mito_seg_id = mito_query_basal_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_basal['mito_mesh_' + str(mito_query_basal_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_basal)

var_iterator_apical = {}
for i in range(len(mito_query_apical_mitolist)):
    mito_seg_id = mito_query_apical_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_apical['mito_mesh_' + str(mito_query_apical_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_apical)

var_iterator_axonal = {}
for i in range(len(mito_query_axonal_mitolist)):
    mito_seg_id = mito_query_axonal_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_axonal['mito_mesh_' + str(mito_query_axonal_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_axonal)

var_iterator_somatic = {}
for i in range(len(mito_query_somatic_mitolist)):
    mito_seg_id = mito_query_somatic_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_somatic['mito_mesh_' + str(mito_query_somatic_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_somatic)

var_iterator_unknown = {}
for i in range(len(mito_query_unknown_mitolist)):
    mito_seg_id = mito_query_unknown_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_unknown['mito_mesh_' + str(mito_query_unknown_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_unknown)

var_iterator_unkdend = {}
for i in range(len(mito_query_unkdend_mitolist)):
    mito_seg_id = mito_query_unkdend_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_unkdend['mito_mesh_' + str(mito_query_unkdend_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_unkdend)


In [27]:
len(var_iterator_apical)

0

In [28]:
# var_iterator_apical

In [31]:
# mito_query_apical_mitolist[0]

In [30]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.075 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_basal = 0.25
mito_color_basal = (0, 0.75, 1) # sky blue
mito_opac_apical = 0.75
mito_color_apical = (0.9, 0.9, 0) # gold 
mito_opac_axonal = 1
mito_color_axonal = (0, 0, 0) # black
mito_opac_somatic = 0.25
mito_color_somatic = (0.7, 0.4, 0.7) # purple
mito_opac_unknown = 1
mito_color_unknown = (1, 0, 0) # red
mito_opac_unkndend = 1
mito_color_unkdend = (0.9, 0.6, 0) # orange

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes
actor_iterator_basal = {}
for i in range(len(mito_query_basal_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_basal['mito_mesh_actor_' + str(mito_query_basal_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_basal['mito_mesh_'+str(mito_query_basal_mitolist[i])], opacity=mito_opac_basal, color=(mito_color_basal))
locals().update(actor_iterator_basal)

actor_iterator_apical = {}
for i in range(len(mito_query_apical_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_apical['mito_mesh_actor_' + str(mito_query_apical_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_apical['mito_mesh_'+str(mito_query_apical_mitolist[i])], opacity=mito_opac_apical, color=(mito_color_apical))
locals().update(actor_iterator_apical)

actor_iterator_axonal = {}
for i in range(len(mito_query_axonal_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_axonal['mito_mesh_actor_' + str(mito_query_axonal_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_axonal['mito_mesh_'+str(mito_query_axonal_mitolist[i])], opacity=mito_opac_axonal, color=(mito_color_axonal))
locals().update(actor_iterator_axonal)

actor_iterator_somatic = {}
for i in range(len(mito_query_somatic_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_somatic['mito_mesh_actor_' + str(mito_query_somatic_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_somatic['mito_mesh_'+str(mito_query_somatic_mitolist[i])], opacity=mito_opac_somatic, color=(mito_color_somatic))
locals().update(actor_iterator_somatic)

actor_iterator_unknown = {}
for i in range(len(mito_query_unknown_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_unknown['mito_mesh_actor_' + str(mito_query_unknown_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_unknown['mito_mesh_'+str(mito_query_unknown_mitolist[i])], opacity=mito_opac_unknown, color=(mito_color_unknown))
locals().update(actor_iterator_unknown)

actor_iterator_unkdend = {}
for i in range(len(mito_query_unkdend_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_unkdend['mito_mesh_actor_' + str(mito_query_unkdend_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_unkdend['mito_mesh_'+str(mito_query_unkdend_mitolist[i])], opacity=mito_opac_unkdend, color=(mito_color_unkdend))
locals().update(actor_iterator_unkdend)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_basal)
actor_iterator.update(actor_iterator_apical)
actor_iterator.update(actor_iterator_axonal)
actor_iterator.update(actor_iterator_somatic)
actor_iterator.update(actor_iterator_unknown)
actor_iterator.update(actor_iterator_unkdend)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000253F1ADE650) at 0x00000253A36C5108>

In [32]:
actor_iterator

{'mesh_actor': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB3170) at 0x00000253F418F228>,
 'mito_mesh_actor_3859902': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB4DE0) at 0x00000253F418F8E8>,
 'mito_mesh_actor_3649414': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB6820) at 0x00000253F418F588>,
 'mito_mesh_actor_3624223': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB3C60) at 0x00000253F418F5E8>,
 'mito_mesh_actor_3088275': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB5470) at 0x00000253F418F108>,
 'mito_mesh_actor_3860288': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB40C0) at 0x00000253F418F528>,
 'mito_mesh_actor_3202039': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB5D30) at 0x00000253F418F048>,
 'mito_mesh_actor_3767159': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x00000253F8BB6EB0) at 0x00000253F418F168>,
 'mito_mesh_actor_2418114': <vtkmodules.vtkRenderingOpenGL2.v

In [33]:
#type(actor_iterator.values())

In [34]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [35]:
cellid

648518346349539100

In [36]:
#pyr_list

# Neuroglancer visualizer

In [37]:
lookup = cellid

In [38]:
query = mito[mito["cellid"] == lookup]

In [39]:
len(query)

531

In [40]:
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349539100%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_19

In [41]:
query

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
35,3417448,33340,103816,62974,1608,103760,62878,1601,103894,63146,1620,648518346349539100,415264,251896,64320,0.747189,9.042512,Somatic
385,2866447,5392,92254,61260,1570,92226,61242,1568,92292,61280,1574,648518346349539100,369016,245040,62800,0.140131,2.280526,Unknown
694,3411689,16104,102770,61410,1719,102696,61276,1715,102848,61552,1722,648518346349539100,411080,245640,68760,0.406888,6.258737,Somatic
1443,3521404,544628,105400,61336,1423,104128,61220,1408,106056,61418,1456,648518346349539100,421600,245344,56920,7.337689,32.092688,Somatic
1988,3859902,259368,111770,61294,1754,111430,61088,1742,111982,61534,1768,648518346349539100,447080,245176,70160,2.894681,8.687554,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209798,3738582,634508,108970,56552,2016,108832,56190,1895,109148,56876,2157,648518346349539100,435880,226208,80640,7.038631,20.869818,Unknown
209869,4080080,645992,114980,55468,1224,114210,55274,1191,115868,55590,1265,648518346349539100,459920,221872,48960,6.361307,14.863219,Unknown
210207,3410791,706696,102232,60314,1482,101802,60064,1465,102602,60588,1506,648518346349539100,408928,241256,59280,6.691491,14.455452,Somatic
210521,3416663,807256,103220,62292,1400,102948,61622,1279,103630,62890,1498,648518346349539100,412880,249168,56000,9.823682,35.053269,Somatic


In [42]:
query.sort_values(by='mito_vx', ascending=False)

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
29472,1452999,6274288,79456,57454,1580,67776,53814,1513,92092,61680,1636,648518346349539100,317824,229816,63200,72.154369,229.925522,Unknown
17931,2445855,5416184,91810,72054,1231,84884,66526,948,99096,76120,1371,648518346349539100,367240,288216,49240,57.996182,160.228430,Basal
5022,1210754,4281408,73226,50160,322,63978,47728,204,81268,50936,538,648518346349539100,292904,200640,12880,49.218792,156.728437,Unknown
18813,3267298,4046352,101908,47092,633,100708,44386,21,103180,51350,1233,648518346349539100,407632,188368,25320,44.409889,128.896140,Unknown
4616,2161989,4032384,87446,55306,1643,79816,53894,1484,93586,56114,1847,648518346349539100,349784,221224,65720,48.202137,165.959954,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17012,3309060,2336,100480,61950,1503,100466,61916,1499,100502,61968,1508,648518346349539100,401920,247800,60120,0.091322,3.362825,Somatic
16963,2245196,2324,81734,42418,217,81720,42386,212,81754,42446,222,648518346349539100,326936,169672,8680,0.086747,2.912149,Unknown
16385,3417248,2204,103598,62746,1534,103578,62726,1532,103624,62760,1538,648518346349539100,414392,250984,61360,0.078072,2.360419,Somatic
15847,3410881,2096,101930,60590,1468,101900,60554,1465,101952,60624,1471,648518346349539100,407720,242360,58720,0.077584,2.561315,Somatic


# Center vtk view at centroid of mitoid of interest

In [43]:
mitoid_query = 3304526

In [44]:
pull_centroid = mito_querydf.reset_index()
pull_centroid = pull_centroid.index[pull_centroid['mito_id'] == mitoid_query].to_list()
pull_centroid[0]

266

In [45]:
mito_mesh_centroid_dict = {}
mito_seg_id = mitoid_query
mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
mito_mesh_centroid_dict['mito_mesh_centroid_' + str(pull_centroid[0])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(mito_mesh_centroid_dict)

In [46]:
mito_mesh_centroid_dict

{'mito_mesh_centroid_266': <trimesh.Mesh(vertices.shape=(5673, 3), faces.shape=(11346, 3))>}

In [47]:
mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid

array([408142.30787598, 239345.5288446 ,  61429.61363441])

In [48]:
backoff_value = 20

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000253A4CD1BB0) at 0x00000253F45A4F48>

In [49]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'_'+str(mitoid_query)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )